In [1]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import time
import datetime as dt
import itertools
import re
import math
from collections import Counter
import operator

#scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#modelling
from catboost import CatBoostClassifier, Pool

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import preprocessing_Origins, is_day, is_WG_open, openDevice, cleaning_processing, FB_process, FB_daily
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


2


# Define parameters

In [3]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
dico_matching = config.dico_matching
nbr_maxdur2beremoved = config.nbr_maxdur2beremoved
dico_garden_opening_hour = config.dico_garden_opening_hour
date_first_opening_WG = config.date_first_opening_WG
print(id_run)
path_extracted_data_visual = os.path.join(path_extracted_data,'visual')
pal_ = config.pal_
close_dates = config.close_dates

#feature for model
li_cont_select = config.li_cont_select
li_bin = config.li_bin
li_cat_select = config.li_cat_select

#create a director if not existing
if not os.path.exists(path_extracted_data_visual):
    os.makedirs(path_extracted_data_visual)
    
path_extracted_data_cleaning_model = os.path.join(path_extracted_data,'Cleaning','model')  

change the configuration file if not done yet!
ALLDATA_


df_LF = pd.read_csv(os.path.join(path_extracted_data, id_run+'_LFCounterEqual0.csv'), parse_dates=['date'], dayfirst=True, sep=';') 
#x0 = df_daily.shape[0]
#fillnan with big number to it also means weird values if there is only nan
#df_daily['date_2remove_weirdLFcounter'] = df_daily.apply(lambda x: x['level'] in df_LF[(df_LF['LFCounter_nbr_equal0']>=10)&\
#(df_LF['sender']==x['TagID'])]['date'].tolist(), axis=1)
print(df_LF.shape)
display(df_LF.head(5))
display(df_LF.tail(5))
df_LF[df_LF['LFCounter_nbr_equal0']>0]

## Preprocess - add HenID,PenID

In [6]:
START_TIME = time.perf_counter()
p = glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\TagUpdates\log*'))
#print('There is %d log files:\n  %s'%(len(p),'  \n  '.join(p)))
df = preprocessing_Origins(p, config, save=True, dodevice=True)
print(df.shape)
display(df.head(2))
display(df.tail(2))
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))
#--> saves a datafarme named: _PreprocessRecords_forcleaning

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:2: DeprecationWarning: invalid escape sequence \T
<>:2: DeprecationWarning: invalid escape sequence \T
<ipython-input-6-4584d3017de0>:2: DeprecationWarning: invalid escape sequence \T
  p = glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\TagUpdates\log*'))


From the focalBirdinfo, you have 155 ative tags


  0%|                                                                                           | 0/51 [00:00<?, ?it/s]

-------------------- DEVICE DATA --------------------
process device data...


100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [13:06<00:00, 15.42s/it]


(5103153, 21)
(46566, 4)
(46566, 7)
(7274527, 15)


,Timestamp,HenID,Zone,PenID,log_file_name,ts_order,TagID,signalstrength,system,time,date,zone2,signalstzone2,zone3,zone4
0,2020-09-29 09:07:00,hen_71,3_Zone,pen12,log_00000001,0,tag_162,10,10 - 12,09:07:00,2020-09-29,2,8,1,0
1,2020-09-29 09:08:12,hen_7,3_Zone,pen11,log_00000001,1,tag_140,10,10 - 12,09:08:12,2020-09-29,2,8,1,0


,Timestamp,HenID,Zone,PenID,log_file_name,ts_order,TagID,signalstrength,system,time,date,zone2,signalstzone2,zone3,zone4
7274525,2021-07-25 23:56:18,hen_217,3_Zone,pen8,log_00000005,124607,tag_75,16,8 - 9,23:56:18,2021-07-25,4,9,3,1
7274526,2021-07-25 23:56:18,hen_173,3_Zone,pen8,log_00000005,124608,tag_86,16,8 - 9,23:56:18,2021-07-25,4,11,3,1


Total running time: 27.35 mn


# Model-based cleaning

In [7]:
#for being more reproductible, we open the file that was saved from cleaning
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_PreprocessRecords_forcleaning.csv'), sep=';', 
                 parse_dates=['Timestamp', 'date'], dayfirst=True) #index_col=0) 
#remove the entries that last less than a second
print(df.shape)
df = df.drop_duplicates(subset=['Timestamp','HenID'], keep='last')
print(min(df['date'].tolist()), max(df['date'].tolist()))
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(7274527, 16)
2020-09-29 00:00:00 2021-07-25 00:00:00
(7261563, 16)


,Unnamed: 0,Timestamp,HenID,Zone,PenID,log_file_name,ts_order,TagID,signalstrength,system,time,date,zone2,signalstzone2,zone3,zone4
0,0,2020-09-29 09:07:00,hen_71,3_Zone,pen12,log_00000001,0,tag_162,10,10 - 12,09:07:00,2020-09-29,2,8,1,0
1,1,2020-09-29 09:08:12,hen_7,3_Zone,pen11,log_00000001,1,tag_140,10,10 - 12,09:08:12,2020-09-29,2,8,1,0
2,2,2020-09-29 09:08:26,hen_71,3_Zone,pen12,log_00000001,2,tag_162,11,10 - 12,09:08:26,2020-09-29,8,10,3,1


In [8]:
#use same step as in cleaning-validation-effect paper
START_TIME = time.perf_counter()
df = cleaning_processing(df=df, date_min=dt.datetime(2020, 9, 29), date_max=dt.datetime(2021, 7, 26), config=config)
df['time'] = df['Timestamp'].map(lambda x: dt.datetime.time(x))
df.rename(columns={'Zone':'Trackingsystem_Zone'}, inplace=True) #thats how it was named in the model 
#add other variables
df['is_dur_bigger_'+str(nbr_maxdur2beremoved)] = df['duration'].map(lambda x: x>=nbr_maxdur2beremoved)
display(df['is_dur_bigger_'+str(nbr_maxdur2beremoved)].value_counts(normalize=True))
df['is_dur_smaller_60sec'] = df['duration'].map(lambda x: x<60)
display(df['is_dur_smaller_60sec'].value_counts(normalize=True))
print('saving...')
df.to_csv(os.path.join(path_extracted_data,'df_recordsmodelpreprocessing.csv'), sep=';', index=False)
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))
print(df.shape)
df.head(3)

(7261563, 16)


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(7261563, 16)


100%|████████████████████████████████████████████████████████████████████████████████| 227/227 [10:43<00:00,  2.84s/it]


small VERIFICATION of zone matching, before droping the old names


...

,Unnamed: 0,Timestamp,HenID,Zone,PenID,ts_order,TagID,signalstrength,system,time,...,next3_zone,previous3_zone,next4_zone,previous4_zone,next5_zone,previous5_zone,previousZone==NextZone,zone2_match,zone3_match,zone4_match
171,171,2020-09-29 09:47:53,hen_47,2_Zone,pen12,104,tag_175,6,10 - 12,09:47:53,...,2_Zone,NaN,2_Zone,NaN,2_Zone,NaN,True,2_Zone,NoOtherZone,NoOtherZone
197,197,2020-09-29 09:51:02,hen_55,2_Zone,pen12,118,tag_167,6,10 - 12,09:51:02,...,2_Zone,NaN,2_Zone,NaN,2_Zone,NaN,True,2_Zone,NoOtherZone,NoOtherZone
346,346,2020-09-29 10:15:40,hen_53,2_Zone,pen12,148,tag_169,6,10 - 12,10:15:40,...,2_Zone,2_Zone,2_Zone,2_Zone,2_Zone,3_Zone,True,2_Zone,NoOtherZone,NoOtherZone


False    0.911787
True     0.088213
Name: is_dur_bigger_900, dtype: float64

True     0.500013
False    0.499987
Name: is_dur_smaller_60sec, dtype: float64

saving...
Total running time: 41.17 mn
(7261563, 44)


,Unnamed: 0,Timestamp,HenID,Trackingsystem_Zone,PenID,ts_order,TagID,signalstrength,system,time,...,previous5_zone,previousZone==NextZone,zone2_match,zone3_match,zone4_match,zone3_match_exist,next2zone==Zone,previous2zone==Zone,is_dur_bigger_900,is_dur_smaller_60sec
0,0,2020-09-29 09:07:00,hen_71,3_Zone,pen12,0,tag_162,10,10 - 12,09:07:00,...,NaN,False,3_Zone,2_Zone,NoOtherZone,1,1,0,False,False
1,1,2020-09-29 09:08:12,hen_7,3_Zone,pen11,1,tag_140,10,10 - 12,09:08:12,...,NaN,False,3_Zone,2_Zone,NoOtherZone,1,1,0,False,False
2,2,2020-09-29 09:08:26,hen_71,3_Zone,pen12,2,tag_162,11,10 - 12,09:08:26,...,NaN,True,miniZone3,4_Zone,2_Zone,1,1,0,False,False


In [9]:
#download model
model = CatBoostClassifier()
path_ = r'G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\DataOutput\TrackingSystem\CleaningAnalysis_\Cleaning\model'
model.load_model(os.path.join(path_,'FINAL_Catboost'));

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
#for more flexibility we open the file that was saved previously
#df = pd.read_csv(os.path.join(path_extracted_data, 'df_recordsmodelpreprocessing.csv'), sep=';', 
#                 parse_dates=['Timestamp','date'], dayfirst=True) 
df['is_day'] = df['Timestamp'].map(lambda x: is_day(x, config.dico_night_hour))
df['is_dur_smaller_60sec'] = df['duration'].map(lambda x: x<60)
display(df['is_dur_smaller_60sec'].value_counts())
df['duration_bounded_mn'] = df['duration'].map(lambda x: min(x,60*60)/60)
df['next_duration_bounded_mn'] = df['next_duration'].map(lambda x: min(x,60*60)/60)
df['previous_duration_bounded_mn'] = df['previous_duration'].map(lambda x: min(x,60*60)/60)
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


True     3630877
False    3630686
Name: is_dur_smaller_60sec, dtype: int64

(7261563, 48)


,Unnamed: 0,Timestamp,HenID,Trackingsystem_Zone,PenID,ts_order,TagID,signalstrength,system,time,...,zone4_match,zone3_match_exist,next2zone==Zone,previous2zone==Zone,is_dur_bigger_900,is_dur_smaller_60sec,is_day,duration_bounded_mn,next_duration_bounded_mn,previous_duration_bounded_mn
0,0,2020-09-29 09:07:00,hen_71,3_Zone,pen12,0,tag_162,10,10 - 12,09:07:00,...,NoOtherZone,1,1,0,False,False,True,1.433333,1.183333,0.000000
1,1,2020-09-29 09:08:12,hen_7,3_Zone,pen11,1,tag_140,10,10 - 12,09:08:12,...,NoOtherZone,1,1,0,False,False,True,4.066667,4.866667,0.000000
2,2,2020-09-29 09:08:26,hen_71,3_Zone,pen12,2,tag_162,11,10 - 12,09:08:26,...,2_Zone,1,1,0,False,False,True,1.183333,0.716667,1.433333


df_ = df[(df['Trackingsystem_Zone']!='1_Zone')&\
         (df['HenID']=='hen_143')&(df['date'].isin([dt.datetime(2020,10,3), 
                                                    dt.datetime(2020,10,4)]))].copy()
print(df_.shape)
df_.head(3)

In [11]:
########### handle nan
#add info on nan values in the initial df, in order to replace at the end all prediction based on features involvig at least 
#one nan, by 1
#Indeed, for records with some nan features, we will predict them a 1 all the time its only 0,05% of the time , i.e. when its 
#first or last transition. Indeed, we dont want to loose any records!
df['nbr_nan'] = df.filter(li_cont_select+li_cat_select, axis=1).isnull().sum(axis=1).tolist()
display(df['nbr_nan'].value_counts())
#fillnan for simplicity, but we will remove those rows predictions anyway
#display BEFORE categoricalfeatures
display(df[li_cat_select].head(3))
df[li_cont_select] = df[li_cont_select].fillna(df[li_cont_select].mean().iloc[0])
#display AFTER categoricalfeatures
df[li_cat_select] = df[li_cat_select].fillna(df[li_cat_select].mode().iloc[0])
#show data
display(df[config.li_cont_select].head(3))
display(df[config.li_cat_select].head(3))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    7260590
1        746
2        227
Name: nbr_nan, dtype: int64

,Trackingsystem_Zone,zone2_match,previous1_zone,next1_zone,system
0,3_Zone,3_Zone,NaN,3_Zone,10 - 12
1,3_Zone,3_Zone,NaN,3_Zone,10 - 12
2,3_Zone,miniZone3,3_Zone,3_Zone,10 - 12


,signalstrength,signalstzone2,duration_bounded_mn,next_duration_bounded_mn,previous_duration_bounded_mn,next2zone==Zone,previous2zone==Zone,zone3_match_exist
0,10,8,1.433333,1.183333,0.000000,1,0,1
1,10,8,4.066667,4.866667,0.000000,1,0,1
2,11,10,1.183333,0.716667,1.433333,1,0,1


,Trackingsystem_Zone,zone2_match,previous1_zone,next1_zone,system
0,3_Zone,3_Zone,3_Zone,3_Zone,10 - 12
1,3_Zone,3_Zone,3_Zone,3_Zone,10 - 12
2,3_Zone,miniZone3,3_Zone,3_Zone,10 - 12


In [12]:
#Too slow with large dataset
START_TIME = time.perf_counter()
#predict, and if its wintergarten (i.e. 1_Zone) put =1
#to slow for large dataset, but more readable:
#df['model_prediction'] = df.apply(lambda x: model.predict(pd.DataFrame(x[li_cont_select+\
#                                                                         li_cat_select]).transpose())[0] if\
#                                  x['Trackingsystem_Zone']!='1_Zone' else 1, axis=1)
#predict, and if its wintergarten (i.e. 1_Zone) put =1
df_ = df[df['Trackingsystem_Zone']!='1_Zone'].copy()
predictions = model.predict(df_.filter(li_cont_select+li_cat_select, axis=1))
x0 = df[df['Trackingsystem_Zone']=='1_Zone'].shape[0]
df_['model_prediction'] = predictions
if df_[df_['model_prediction'].isnull()].shape[0]!=0:
    print('ERROR')
    sys.exit()
df = pd.merge(df, df_.filter(['model_prediction','HenID','Timestamp'], axis=1), on=['HenID','Timestamp'], how='left')
print(df[df['model_prediction'].isnull()].shape)
if df[df['model_prediction'].isnull()].shape[0]!=x0:
    print('ERROR: you have some nan in your predictions')
    sys.exit()
display(df['model_prediction'].value_counts(normalize=True))

#a bit more conservative than in the cleaning, as we have now longer durations, and we dont want to be able to add big errors

#predict false if <60seconds in WG or at night, else true
df.loc[(df['is_dur_smaller_60sec'])&((df['Trackingsystem_Zone']=='1_Zone')|(~df['is_day'])), 'model_prediction'] = 0
#always =1 if there was some nan values
display(df['model_prediction'].value_counts(normalize=True)) #in testing it was 4,9% of wrong records: 42/(42+821)*100)

#always =1 if >15mn or if there was some NAN values: correct if >15 as we saw its ~always the case, when not its because the  
#hen went in the zone that was wrongly registered and which was the latest registrations of that hen
df.loc[(df['is_dur_bigger_'+str(nbr_maxdur2beremoved)])|(df['nbr_nan']!=0), 'model_prediction'] = 1
display(df['model_prediction'].value_counts(normalize=True)) #in testing it was 4,9% of wrong records: 42/(42+821)*100)

#predict=1 if nan
df.loc[df['nbr_nan']!=0, 'model_prediction'] = 1
display(df['model_prediction'].value_counts(normalize=True)) #in testing it was 4,9% of wrong records: 42/(42+821)*100)

END_TIME = time.perf_counter()
print ("Total running time for prediction: %.2f mn" %((END_TIME-START_TIME)/60))
display(display(df.head(3)))
#prediction_proba = model.predict_proba(df[df['Trackingsystem_Zone']!='1_Zone'])
#df['model_prediction_proba0'] = [round(x[0],2) for x in prediction_proba]
#df['model_prediction_proba1'] = [round(x[1],2) for x in prediction_proba]

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(180037, 50)


1.0    0.952246
0.0    0.047754
Name: model_prediction, dtype: float64

1.0    0.936247
0.0    0.063753
Name: model_prediction, dtype: float64

1.0    0.936821
0.0    0.063179
Name: model_prediction, dtype: float64

1.0    0.936821
0.0    0.063179
Name: model_prediction, dtype: float64

Total running time for prediction: 0.50 mn


,Unnamed: 0,Timestamp,HenID,Trackingsystem_Zone,PenID,ts_order,TagID,signalstrength,system,time,...,next2zone==Zone,previous2zone==Zone,is_dur_bigger_900,is_dur_smaller_60sec,is_day,duration_bounded_mn,next_duration_bounded_mn,previous_duration_bounded_mn,nbr_nan,model_prediction
0,0,2020-09-29 09:07:00,hen_71,3_Zone,pen12,0,tag_162,10,10 - 12,09:07:00,...,1,0,False,False,True,1.433333,1.183333,0.000000,1,1.0
1,1,2020-09-29 09:08:12,hen_7,3_Zone,pen11,1,tag_140,10,10 - 12,09:08:12,...,1,0,False,False,True,4.066667,4.866667,0.000000,1,1.0
2,2,2020-09-29 09:08:26,hen_71,3_Zone,pen12,2,tag_162,11,10 - 12,09:08:26,...,1,0,False,False,True,1.183333,0.716667,1.433333,0,1.0


None

In [13]:
#will take TIME to save
#Note that we cant use the duration info anymore so we are filtering it
df.filter(['HenID','PenID','system','Trackingsystem_Zone','model_prediction','Timestamp','date'],
          axis=1).to_csv(os.path.join(path_extracted_data, id_run+'_Rawdata&Model.csv'), index=False, sep=';')

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Remove filtered records & consecutives equal zones

In [14]:
#for being more reproductible, we open the file that was saved from cleaning
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_Rawdata&Model.csv'), sep=';', parse_dates=['Timestamp','date'],
                 dayfirst=True) 
#remove predicted-wrong-records
print(df.shape)
print(df[df['model_prediction'].isnull()].shape)
df = df[df['model_prediction']!=0].copy()
print(df.shape)
#simply rename the tracking system zone into the model zone :) 
df.rename(columns={'Trackingsystem_Zone':'Zone'}, inplace=True)
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(7261563, 7)
(108959, 7)
(6809667, 7)
(6809667, 7)


,HenID,PenID,system,Zone,model_prediction,Timestamp,date
0,hen_71,pen12,10 - 12,3_Zone,1.0,2020-09-29 09:07:00,2020-09-29
1,hen_7,pen11,10 - 12,3_Zone,1.0,2020-09-29 09:08:12,2020-09-29
2,hen_71,pen12,10 - 12,3_Zone,1.0,2020-09-29 09:08:26,2020-09-29


In [15]:
#######################################################################################################################
#ceiz = consecutives equal initial zone
print('-----------------------------------------------------------------------------------------------')
print('remove consecutives equal zone for the same hen.........')
print('-----------------------------------------------------------------------------------------------')

li_df = []
#more efficient to do it per hen, as it wont need to search in the whole dataframe, and we can simply shift the timestamp column
for i, df_hen in tqdm.tqdm(df.groupby(['HenID'])):
    #as the next record date (sort by date, then simply shift by one row and add nan at then end)
    df_hen = df_hen.sort_values(['Timestamp'], ascending=True)
    #same date, one must take the last recorded one & sorting by date might change it. Also it already should be sorted by date
    df_hen['next_record_date'] = df_hen['Timestamp'].tolist()[1:]+[np.nan]
    #compute duration
    df_hen['duration'] = df_hen.apply(lambda x: x['next_record_date']-x['Timestamp'], axis=1)
    #compute the last record date in order to put interzone also when the duration is >=nbr_sec_flickering1
    df_hen['previous_record_date'] = [np.nan]+df_hen['Timestamp'].tolist()[0:-1]
    #compute previous duration in order to put interzone also when the duration is >=nbr_sec_flickering1
    df_hen['previous_duration'] = [np.nan]+df_hen['duration'].tolist()[0:-1]
    #add next record for the impossible movement
    df_hen['next_zone'] = df_hen['Zone'].tolist()[1:]+[np.nan]
    #add previous record for the consecutives equal initial zones
    df_hen['previous_zone'] = [np.nan]+df_hen['Zone'].tolist()[0:-1]
    df_hen['previous_previous_zone'] = [np.nan]+df_hen['previous_zone'].tolist()[0:-1]
    li_df.append(df_hen)
#put again in one dataframe
df = pd.concat(li_df)
#dont care about the false positive warning   

#True if next zone is equal to the actual zone
df['correction_is_consecutive_equal_initial_zone'] = False
#if the previous zone is the same, then its cez that should be removed
df.loc[df['previous_zone']==df['Zone'], 'correction_is_consecutive_equal_initial_zone'] = True


################# save
df = df.sort_values(['Timestamp'], ascending=True)
#save to verify CEIZ
df.to_csv(os.path.join(path_extracted_data, id_run+'_records_GeneralCleaning_VERIFICATION_CEIZ.csv'), sep=';', index=False) 
df = df[~(df['correction_is_consecutive_equal_initial_zone'])]

-----------------------------------------------------------------------------------------------
remove consecutives equal zone for the same hen.........
-----------------------------------------------------------------------------------------------


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|████████████████████████████████████████████████████████████████████████████████| 227/227 [04:18<00:00,  1.14s/it]


In [16]:
#small verification
df_hen[['Timestamp','HenID','previous_zone','Zone']].head(10)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Timestamp,HenID,previous_zone,Zone
1942,2020-09-29 14:33:02,hen_99,NaN,3_Zone
1982,2020-09-29 14:38:10,hen_99,3_Zone,2_Zone
2147,2020-09-29 14:58:57,hen_99,2_Zone,3_Zone
2189,2020-09-29 15:03:42,hen_99,3_Zone,3_Zone
2192,2020-09-29 15:03:58,hen_99,3_Zone,3_Zone
2201,2020-09-29 15:04:31,hen_99,3_Zone,3_Zone
2288,2020-09-29 15:15:46,hen_99,3_Zone,2_Zone
3407,2020-09-29 16:54:27,hen_99,2_Zone,3_Zone
3417,2020-09-29 16:55:54,hen_99,3_Zone,3_Zone
3421,2020-09-29 16:56:36,hen_99,3_Zone,3_Zone


## Remove records in WG when WG was close

In [17]:
df['is_WG_open'] = df['Timestamp'].map(lambda x: is_WG_open(x, config, epsi_open=0, epsi_close=20))
#we keep the epsi_open=0 as the system appeared to always open between 20sec of 2mn AFTER the indicated time (from video 
#observation)
#we set epsi_close=20 as for video observation we saw that itwoul dbe between + and -15 mn than the indicated time. So we added
#some flexibility by puting 20mn.
print(df['is_WG_open'].value_counts(normalize=True))
display(df[(df['Zone']=='1_Zone')&(df['is_WG_open'])][['Timestamp','is_WG_open']].head(5))
display(df[(df['Zone']=='1_Zone')&(~df['is_WG_open'])][['Timestamp','is_WG_open']].head(5))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


True     0.550862
False    0.449138
Name: is_WG_open, dtype: float64


,Timestamp,is_WG_open
48777,2020-10-08 11:01:14,True
48805,2020-10-08 11:05:21,True
48808,2020-10-08 11:06:29,True
48841,2020-10-08 11:09:54,True
48871,2020-10-08 11:11:42,True


,Timestamp,is_WG_open
30628,2020-10-05 11:58:13,False
30639,2020-10-05 11:59:40,False
30836,2020-10-05 12:27:06,False
30892,2020-10-05 12:34:59,False
30912,2020-10-05 12:37:56,False


In [18]:
#remove WG when close
print(df.shape)
df = df[~((~df['is_WG_open'])&(df['Zone']=='1_Zone'))]
print(df.shape)

(2704970, 16)


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2695668, 16)


df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_CLEANEDDATA.csv'), sep=';', parse_dates=['Timestamp', 'date'],
                dayfirst=True) 
print(df.shape)
df.head(3)

# remove transitions smaller than 15mn over night

In [22]:
########### remove transitions smaller than 15mn over night
#compute duration again as is not updated with alst cleaning
li_df = []
#more efficient to do it per hen, as it wont need to search in the whole dataframe, and we can simply shift the timestamp column
for i, df_hen in tqdm.tqdm(df.groupby(['HenID'])):
    #as the next record date (sort by date, then simply shift by one row and add nan at then end)
    df_hen = df_hen.sort_values(['Timestamp'], ascending=True)
    #same date, one must take the last recorded one & sorting by date might change it. Also it already should be sorted by date
    df_hen['next_record_date'] = df_hen['Timestamp'].tolist()[1:]+[np.nan]
    #compute duration
    df_hen['duration_mn'] = df_hen.apply(lambda x: (x['next_record_date']-x['Timestamp']).total_seconds()/60, axis=1)
    li_df.append(df_hen)
#put again in one dataframe
df = pd.concat(li_df)
df['is_day'] = df['Timestamp'].map(lambda x: is_day(x, config.dico_night_hour))
#display(df[~df['is_day']].head(3))
#df[~df['is_day']].tail(3)
print(df.shape)
display(df.head(3))
#df[(df['duration_mn']<15) & (df['is_day']==False)]
#df[(df['HenID']=='hen_1')&(df['Timestamp']<dt.datetime(2020,10,1,10,1,1))&(df['Timestamp']>dt.datetime(2020,9,30,17,50,1))]
df = df[~((df['duration_mn']<15) & (df['is_day']==False))]
print(df.shape)
df.head(3)

100%|████████████████████████████████████████████████████████████████████████████████| 227/227 [01:36<00:00,  2.35it/s]


(2695015, 18)


,HenID,PenID,system,Zone,model_prediction,Timestamp,date,next_record_date,duration,previous_record_date,previous_duration,next_zone,previous_zone,previous_previous_zone,correction_is_consecutive_equal_initial_zone,is_WG_open,duration_mn,is_day
56,hen_1,pen9,8 - 9,2_Zone,1.0,2020-09-29 09:50:00,2020-09-29,2020-09-29 17:56:25,0 days 08:06:25,NaN,NaN,3_Zone,NaN,NaN,False,False,486.416667,True
1026,hen_1,pen9,8 - 9,3_Zone,1.0,2020-09-29 17:56:25,2020-09-29,2020-09-30 09:55:59,0 days 02:52:35,2020-09-29 09:50:00,0 days 08:06:25,3_Zone,2_Zone,NaN,False,False,959.566667,False
1363,hen_1,pen9,8 - 9,5_Zone,1.0,2020-09-30 09:55:59,2020-09-30,2020-09-30 11:34:30,0 days 01:32:11,2020-09-30 09:51:57,0 days 00:04:02,5_Zone,3_Zone,3_Zone,False,False,98.516667,True


(2681720, 18)


,HenID,PenID,system,Zone,model_prediction,Timestamp,date,next_record_date,duration,previous_record_date,previous_duration,next_zone,previous_zone,previous_previous_zone,correction_is_consecutive_equal_initial_zone,is_WG_open,duration_mn,is_day
56,hen_1,pen9,8 - 9,2_Zone,1.0,2020-09-29 09:50:00,2020-09-29,2020-09-29 17:56:25,0 days 08:06:25,NaN,NaN,3_Zone,NaN,NaN,False,False,486.416667,True
1026,hen_1,pen9,8 - 9,3_Zone,1.0,2020-09-29 17:56:25,2020-09-29,2020-09-30 09:55:59,0 days 02:52:35,2020-09-29 09:50:00,0 days 08:06:25,3_Zone,2_Zone,NaN,False,False,959.566667,False
1363,hen_1,pen9,8 - 9,5_Zone,1.0,2020-09-30 09:55:59,2020-09-30,2020-09-30 11:34:30,0 days 01:32:11,2020-09-30 09:51:57,0 days 00:04:02,5_Zone,3_Zone,3_Zone,False,False,98.516667,True


In [24]:
df.head(2)

,HenID,PenID,system,Zone,model_prediction,Timestamp,date,next_record_date,duration,previous_record_date,previous_duration,next_zone,previous_zone,previous_previous_zone,correction_is_consecutive_equal_initial_zone,is_WG_open,duration_mn,is_day
56,hen_1,pen9,8 - 9,2_Zone,1.0,2020-09-29 09:50:00,2020-09-29,2020-09-29 17:56:25,0 days 08:06:25,NaN,NaN,3_Zone,NaN,NaN,False,False,486.416667,True
1026,hen_1,pen9,8 - 9,3_Zone,1.0,2020-09-29 17:56:25,2020-09-29,2020-09-30 09:55:59,0 days 02:52:35,2020-09-29 09:50:00,0 days 08:06:25,3_Zone,2_Zone,NaN,False,False,959.566667,False


# Save

In [25]:
print(df.shape)
df.head(3)
df.to_csv(os.path.join(path_extracted_data, id_run+'_CLEANEDDATA.csv'), sep=';', index=False) 

(2681720, 18)


## MLP comparison (raw vs cleaned)

In [26]:
#for being more reproductible, we open the file that was saved from cleaning
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_Rawdata&Model.csv'), sep=';', parse_dates=['Timestamp','date'],
                 dayfirst=True) 
print(df.shape)
df.head(3)

(7261563, 7)


,HenID,PenID,system,Trackingsystem_Zone,model_prediction,Timestamp,date
0,hen_71,pen12,10 - 12,3_Zone,1.0,2020-09-29 09:07:00,2020-09-29
1,hen_7,pen11,10 - 12,3_Zone,1.0,2020-09-29 09:08:12,2020-09-29
2,hen_71,pen12,10 - 12,3_Zone,1.0,2020-09-29 09:08:26,2020-09-29


In [27]:
df['model_prediction'].value_counts(normalize=True)

1.0    0.936821
0.0    0.063179
Name: model_prediction, dtype: float64

In [28]:
START_TIME = time.perf_counter()
#choose specific dates to plot, November is better as hens starts to move more and the algo have never seen it
dmin = dt.datetime(2021,5,10)
dmax = dt.datetime(2021,5,20)
#df_ = df[(df['date']<dt.datetime(2020,11,16))&(df['date']>dt.datetime(2020,11,10))]
df_ = df[(df['date']<=dmax)&(df['date']>=dmin)].copy()
print(df_.shape)
#only print the hens with at least one transition per day (simplicity and interestignly)
#to sort the yaxis
dico_zone_order = {'1_Zone':0, '2_Zone':1, '3_Zone':2, '4_Zone':3, '5_Zone':4}
li_ts = ['Trackingsystem_Zone','Model_Zone']
li_date = df_['date'].unique()
#for each batch draw 2 timeseries, the raw one and the model one
for henID, df_plt in tqdm.tqdm(df_.groupby(['HenID'])):
    df_plt = df_plt.sort_values(['Timestamp']).copy()
    c = len(li_ts) ; l = len(li_date)
    fig, ax = plt.subplots(figsize=(c*5, l*1))
    i = 1
    if len(df_plt['date'].unique())==len(li_date):
        mi = min(df_plt['Timestamp'].tolist()) ; ma = max(df_plt['Timestamp'].tolist())
        Daterange = pd.date_range(start = mi+dt.timedelta(seconds=(60-mi.second)), 
                                  end = ma-dt.timedelta(seconds=(ma.second+1)), 
                                  freq = 'S') 
        df_plt_ = df_plt.copy()
        df_plt_.set_index('Timestamp', inplace=True)
        df_plt_ = df_plt_.reindex(Daterange, method='ffill').reset_index()
        df_plt_.rename(columns={'index':'Timestamp'}, inplace=True)
        #add date again, as the reindexing also extended the date
        df_plt_['date'] = df_plt_['Timestamp'].map(lambda x: dt.datetime.date(x))
        #remove first & last date
        df_plt_ = df_plt_[~df_plt_['date'].isin([max(df_plt_['date'].tolist()),min(df_plt_['date'].tolist())])]
        #put xlabel into numbers for the ploting
        df_plt_['Trackingsystem_Zone'] = df_plt_['Trackingsystem_Zone'].map(lambda x: int(dico_zone_order[x]))       
        for d, df_plt__ in df_plt_.groupby(['date']):
            for v in li_ts:
                df_plt___ = df_plt__.copy()
                if v=='Model_Zone':
                    df_plt___ = df_plt___[df_plt___['model_prediction']==1]
                plt.subplot(l,c,i)
                plt.tight_layout(pad=0.3) #add spacing between each plot
                if i%2==1:
                    plt.yticks([0,1,2,3,4], ['Winter garden', 'Litter', 'Lower perch','Nestbox','Top floor'])
                else:
                    plt.yticks([0,1,2,3,4], ['','','','',''])
                if i in [1,2]:
                    plt.title(v.replace('_',' '), size=13)
                i = i+1
                plt.plot(df_plt___['Timestamp'].tolist(), df_plt___['Trackingsystem_Zone'].tolist(), 
                         color=pal_[v], linewidth=1)
                li_hour = pd.date_range(start = d,  end = d+dt.timedelta(days=1), freq = 'H')
                plt.xticks(li_hour , [str(i.hour)+'h' for i in li_hour], fontsize=5)                
                plt.yticks(fontsize=8)
                plt.xlabel(str(d).split('T')[0], size=7)
                plt.ylim(0, 4.2)  
        plt.savefig(os.path.join(path_extracted_data_visual, 
                                 henID+'_'+str(dmin).split(' ')[0]+'_'+str(dmax).split(' ')[0]+'.png'), 
                    format='png', bbox_inches='tight', dpi=300)
    plt.clf()
    plt.close("all")    
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))
#hen4, 22.10.2020: flickering. solved?

(288402, 7)


100%|████████████████████████████████████████████████████████████████████████████████| 158/158 [43:56<00:00, 16.69s/it]

Total running time: 43.95 mn
